STEP 2:

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.


Create a Jupyter Notebook called database_engineering.ipynb and use this to complete all of your Database Engineering work.
Use Pandas to read your cleaned measurements and stations CSV data.
Use the engine and connection string to create a database called hawaii.sqlite.

Use declarative_base and create ORM classes for each table.


You will need a class for Measurement and for Station.
Make sure to define your primary keys.


Once you have your ORM classes defined, create the tables in the database using create_all.


In [178]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date, Text

from sqlalchemy.orm import Session, sessionmaker
import sqlalchemy

import pandas as pd
from datetime import datetime
import time

from sqlalchemy.orm import relationship
from sqlalchemy.ext.hybrid import hybrid_property
from sqlalchemy import select, func



In [179]:
engine=create_engine("sqlite:///hawaii.sqlite")

In [180]:
conn=engine.connect()

In [181]:
Base=declarative_base()

class Prcp(Base):
    __tablename__='prcps'
    
    id=Column(Integer,primary_key=True)
    date=Column(Date)
    prcp=Column(Float)
    tobs=Column(Float)
    
    @hybrid_property
    def datetime(self):
        # @todo: add python parsing of date and time to produce the result
        str_value = self.date
        return datetime.strptime(str_value, "%Y-%m-%d")

    @datetime.expression
    def datetime(Prcp):
        # @note: query specific value
        dt_column =(func.substr(Prcp.date, 1,4) + "-" + func.substr(Prcp.date, 5, 2) + "-" + func.substr(Prcp.date, 6))
        dt_column = func.datetime(dt_column)
        return dt_column
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

class Station(Base):
    __tablename__='stations'
    id=Column(Integer,primary_key=True)
    station=Column(String)
    latitude=Column(Float)
    longitude=Column(Float)
    elevation=Column(Float)
    prcp=Column(Float)
    tobs=Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [182]:
Base.metadata.create_all(engine)

In [183]:
metadata=MetaData(bind=engine)
metadata.reflect()

In [184]:
# data_table1=pd.read_csv('Resources/clean_date_prcp_tobs.csv')
data_table1=pd.read_csv('Resources/clean_date_prcp_tobs.csv',parse_dates=True, index_col='date')
# measures=data_table1.to_dict(orient='records')

data_table2=pd.read_csv('Resources/clean_station_prcp_tobs.csv')
stations_data=data_table2.to_dict(orient='records')

In [185]:
n=pd.DataFrame(data_table1)
n.head()

n=n.reset_index()
type(n.iloc[1,0])

pandas._libs.tslibs.timestamps.Timestamp

In [186]:
n['date']=pd.to_datetime(n['date']).apply(lambda x: x.date())



In [187]:
measures=n.to_dict(orient='records')

In [188]:
print(measures[:5])


[{'date': datetime.date(2010, 1, 1), 'prcp': 0.15142857142857144, 'tobs': 69.71428571428571}, {'date': datetime.date(2010, 1, 2), 'prcp': 0.002857142857142857, 'tobs': 67.0}, {'date': datetime.date(2010, 1, 3), 'prcp': 0.0, 'tobs': 74.0}, {'date': datetime.date(2010, 1, 4), 'prcp': 0.0014285714285714286, 'tobs': 75.0}, {'date': datetime.date(2010, 1, 5), 'prcp': 0.005, 'tobs': 74.5}]


In [136]:
for record in measures:
    record['date']=datetime.strptime(record['date'],'%Y-%m-%d').date()


In [137]:
print(stations_data[:2])

[{'station': 'USC00511918', 'latitude': 21.3152, 'longitude': -157.9992, 'elevation': 0.9, 'prcp': 0.047971014492753664, 'tobs': 71.52743271221532}, {'station': 'USC00513117', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6, 'prcp': 0.14192136498516264, 'tobs': 72.67804154302671}]


In [189]:
measures_table=sqlalchemy.Table('prcps',metadata,autoload=True)

In [139]:
stations_table=sqlalchemy.Table('stations',metadata,autoload=True)

In [190]:
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [191]:
session.execute(measures_table.insert(),measures)


In [175]:
session.close()

In [142]:
session.execute(stations_table.insert(),stations_data)

In [192]:
prcp_instance=Prcp()
session.add(prcp_instance)
session.commit()


In [193]:
session.close()

In [173]:
conn.execute(measures_table.insert(),measures)


In [104]:
conn.execute(stations_table.insert(),stations_data)

In [174]:
conn.execute("select * from prcps limit 10").fetchall()

[(1, '2010-01-01', 0.15142857142857144, 69.71428571428571),
 (2, '2010-01-02', 0.002857142857142857, 67.0),
 (3, '2010-01-03', 0.0, 74.0),
 (4, '2010-01-04', 0.0014285714285714286, 75.0),
 (5, '2010-01-05', 0.005, 74.5),
 (6, '2010-01-06', 0.11600000000000002, 73.4),
 (7, '2010-01-07', 0.275, 70.16666666666667),
 (8, '2010-01-08', 0.008571428571428572, 65.57142857142857),
 (9, '2010-01-09', 0.002857142857142857, 68.57142857142857),
 (10, '2010-01-10', 0.002857142857142857, 72.71428571428571)]

In [84]:
conn.execute("select * from stations limit 10").fetchall()

[(1, 'USC00511918', 21.3152, -157.9992, 0.9, None, 71.52743271221532),
 (2, 'USC00513117', 21.4234, -157.8015, 14.6, None, 72.67804154302671),
 (3, 'USC00514830', 21.5213, -157.8374, 7.0, None, 74.8131130614352),
 (4, 'USC00516128', 21.3331, -157.8025, 152.4, None, 70.86513687600645),
 (5, 'USC00517948', 21.3934, -157.9751, 11.9, None, 74.58711566617863),
 (6, 'USC00518838', 21.4992, -158.0111, 306.6, None, 72.67543859649123),
 (7, 'USC00519281', 21.45167, -157.84888999999995, 32.9, None, 71.66378066378067),
 (8, 'USC00519397', 21.2716, -157.8168, 3.0, None, 74.56424581005588),
 (9, 'USC00519523', 21.33556, -157.71139, 19.5, None, 74.5326594090202),
 (10, 'USC00511918', 21.3152, -157.9992, 0.9, None, 71.52743271221532)]

In [85]:
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [127]:
session.rollback()

In [86]:
session.execute("select * from stations limit 10").fetchall()

[(1, 'USC00511918', 21.3152, -157.9992, 0.9, None, 71.52743271221532),
 (2, 'USC00513117', 21.4234, -157.8015, 14.6, None, 72.67804154302671),
 (3, 'USC00514830', 21.5213, -157.8374, 7.0, None, 74.8131130614352),
 (4, 'USC00516128', 21.3331, -157.8025, 152.4, None, 70.86513687600645),
 (5, 'USC00517948', 21.3934, -157.9751, 11.9, None, 74.58711566617863),
 (6, 'USC00518838', 21.4992, -158.0111, 306.6, None, 72.67543859649123),
 (7, 'USC00519281', 21.45167, -157.84888999999995, 32.9, None, 71.66378066378067),
 (8, 'USC00519397', 21.2716, -157.8168, 3.0, None, 74.56424581005588),
 (9, 'USC00519523', 21.33556, -157.71139, 19.5, None, 74.5326594090202),
 (10, 'USC00511918', 21.3152, -157.9992, 0.9, None, 71.52743271221532)]

In [87]:
session.execute("select*from prcps").fetchall()

[(1, '2010-01-01', 0.15142857142857144, 69.71428571428571),
 (2, '2010-01-02', 0.002857142857142857, 67.0),
 (3, '2010-01-03', 0.0, 74.0),
 (4, '2010-01-04', 0.0014285714285714286, 75.0),
 (5, '2010-01-05', 0.005, 74.5),
 (6, '2010-01-06', 0.11600000000000002, 73.4),
 (7, '2010-01-07', 0.275, 70.16666666666667),
 (8, '2010-01-08', 0.008571428571428572, 65.57142857142857),
 (9, '2010-01-09', 0.002857142857142857, 68.57142857142857),
 (10, '2010-01-10', 0.002857142857142857, 72.71428571428571),
 (11, '2010-01-11', 0.12571428571428572, 66.71428571428571),
 (12, '2010-01-12', 0.002857142857142857, 61.57142857142857),
 (13, '2010-01-13', 0.0033333333333333327, 61.5),
 (14, '2010-01-14', 0.0, 66.42857142857143),
 (15, '2010-01-15', 0.0, 64.83333333333333),
 (16, '2010-01-16', 0.0, 67.33333333333333),
 (17, '2010-01-17', 0.0, 68.0),
 (18, '2010-01-18', 0.0, 73.5),
 (19, '2010-01-19', 0.0, 69.42857142857143),
 (20, '2010-01-20', 0.0014285714285714286, 67.85714285714286),
 (21, '2010-01-21', 0.

In [89]:
prcp_instance=Prcp()

In [90]:
session.add(prcp_instance)

In [91]:
session.commit()

In [ ]:
# from numpy import genfromtxt
# from time import time
# from datetime import datetime
# from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy import create_engine
# from sqlalchemy.orm import sessionmaker

# def Load_Data(file_name):
#     data = genfromtxt(file_name, delimiter=',')# skiprows=1, converters={0: lambda s: str(s)})
#     return data.tolist()

# Base = declarative_base()

# class cdb1(Base):
#     #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
#     __tablename__ = 'cdb1'
#     __table_args__ = {'sqlite_autoincrement': True}
#     #tell SQLAlchemy the name of column and its attributes:
#     id = Column(Integer, primary_key=True, nullable=False) 
#     name = Column(VARCHAR(40))
#     shack = Column(VARCHAR)
#     db = Column(Integer)
#     payments = Column(Integer)
#     status = Column(VARCHAR)


# if __name__ == "__main__":
#     t = time()
#     print 'creating database'

#     #Create the database
#     engine = create_engine('sqlite:///cdb.db')
#     Base.metadata.create_all(engine)

#     #Create the session
#     session = sessionmaker()
#     session.configure(bind=engine)
#     s = session()

#     try:
#         file_name = 'client_db.csv'
#         data = Load_Data(file_name)

#         for i in data:
#             record = cdb1(**{
#                 'name' : i[0],
#                 'shack' : i[1],
#                 'db' : i[2],
#                 'payments' : i[3],
#                 'status' : i[4]
#             })
#             s.add(record) #Add all the records

#         s.commit() #Attempt to commit all the records
#     except:
#         s.rollback() #Rollback the changes on error
#         print 'error in reading'
#     finally:
#         s.close() #Close the connection